# Clustering

In [24]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

encoded_females_sep = pd.read_csv('data/encoded_females_sep.csv',index_col='customer_id')
encoded_males_sep = pd.read_csv('data/encoded_males_sep.csv',index_col='customer_id')
encoded_females_joint = pd.read_csv('data/encoded_females_joint.csv',index_col='customer_id')
encoded_males_joint = pd.read_csv('data/encoded_males_joint.csv',index_col='customer_id')

females = pd.read_csv('data/females.csv',index_col='customer_id')
males = pd.read_csv('data/males.csv',index_col='customer_id')

In [25]:
from sklearn.cluster import KMeans
k = 6
random_state=42
kmeans = KMeans(n_clusters=k, random_state=random_state)

In [26]:
from sklearn.cluster import AgglomerativeClustering
n_clusters = 10
linkage = 'ward'
agglom = AgglomerativeClustering(n_clusters = n_clusters,linkage=linkage)

# Training

In [51]:
np.unique(kmeans.labels_)
np.where(kmeans.labels_ == 4)[0]
#females.iloc[np.where(kmeans.labels_ == 4,kmeans.labels_,None) ,:]
#females.shape,encoded_females_sep.shape, np.where(kmeans.labels_ == 4)

array([    1,     2,     6, ..., 13360, 13362, 13363])

In [53]:
kmeans.fit_predict(encoded_females_sep)
female_sep_labels = kmeans.labels_
female_sep_centroids = np.array([females.iloc[np.where(kmeans.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
female_sep_centroids = pd.DataFrame(female_sep_centroids, columns = females.columns)

kmeans.fit_predict(encoded_males_sep)
male_sep_labels = kmeans.labels_
male_sep_centroids = np.array([males.iloc[np.where(kmeans.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
male_sep_centroids = pd.DataFrame(male_sep_centroids, columns = males.columns)

kmeans.fit_predict(encoded_females_joint)
female_joint_labels = kmeans.labels_
female_joint_centroids = np.array([females.iloc[np.where(kmeans.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
female_joint_centroids = pd.DataFrame(female_joint_centroids, columns = females.columns)

kmeans.fit_predict(encoded_males_joint)
male_joint_labels = kmeans.labels_
male_joint_centroids = np.array([males.iloc[np.where(kmeans.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
male_joint_centroids = pd.DataFrame(male_joint_centroids, columns = males.columns)

In [ ]:
agglom.fit(encoded_females_sep)
agglom_female_sep_labels = agglom.labels_
agglom_female_sep_centroids = np.array([females.iloc[np.where(agglom.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
agglom_female_sep_centroids = pd.DataFrame(female_sep_centroids, columns = females.columns)

agglom.fit(encoded_males_sep)
agglom_male_sep_centroids = np.array([males.iloc[np.where(agglom.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
agglom_male_sep_centroids = pd.DataFrame(male_sep_centroids, columns = males.columns)

agglom.fit(encoded_females_joint)
agglom_female_joint_centroids = np.array([females.iloc[np.where(agglom.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
agglom_female_joint_centroids = pd.DataFrame(female_joint_centroids, columns = females.columns)

agglom.fit(encoded_males_joint)
agglom_male_joint_centroids = np.array([males.iloc[np.where(agglom.labels_ == c)[0],:].mean(axis=0) for c in range(k)])
agglom_male_joint_centroids = pd.DataFrame(male_joint_centroids, columns = males.columns)

# Rough Check of Fit

In [ ]:
print('k-means:', silhouette_score(encoded_females_sep, encoded_females_sep_labels), k)
print('k-means:', silhouette_score(encoded_males_sep, encoded_males_sep_labels), k)
print('k-means:', silhouette_score(encoded_females_joint, encoded_females_joint_labels), k)
print('k-means:', silhouette_score(encoded_males_joint, encoded_males_joint_labels), k)
#print('Agglomerative:', silhouette_score(encoded_females_sep, agglom.labels_))
#print('Agglomerative:', silhouette_score(encoded_males_sep, agglom.labels_))
#print('Agglomerative:', silhouette_score(encoded_females_joint, agglom.labels_))
#print('Agglomerative:', silhouette_score(encoded_males_joint, agglom.labels_))